In [218]:
import numpy as np
import sympy
import cirq
import tensorflow as tf
import tensorflow_quantum as tfq

class CirqSmartSolver:
    def __init__(self, n_qubits=3, observable_name=None, ground_state_energy=None):

        """
        observable_name:: specifies the hamiltonian; can be either string (if in templates, see load_observable function, a list
        or numpy array.

        target_reward:: minus the ground energy (or estimation), used as label for variational optimization.
        """

        self.name = "CirqSolver"
        self.n_qubits = n_qubits
        self.qubits = cirq.GridQubit.rect(1, n_qubits)
        self.observable_name = observable_name

        # Value to use as label for continuous optimization; this appears in variational model
        if ground_state_energy is None:
            self.ground_state_energy = self.n_qubits  # mostly for the ising high transv fields.
        else:
            self.ground_state_energy = ground_state_energy

        self.observable = self.load_observable(observable_name) #Ising hamiltonian with list of pauli_gates

        # Indexed cnots total number n!/(n-2)! = n*(n-1) (if all connections are allowed)
        self.indexed_cnots = {}
        count = 0
        for control in range(self.n_qubits):
            for target in range(self.n_qubits):
                if control != target:
                    self.indexed_cnots[str(count)] = [control, target]
                    count += 1
        self.number_of_cnots = len(self.indexed_cnots)
        #int(np.math.factorial(self.n_qubits)/np.math.factorial(self.n_qubits -2))

        # Create one_hot a+lphabet
        self.alphabet_gates = [cirq.CNOT, cirq.ry, cirq.rx(-np.pi/2), cirq.I]
        self.alphabet = []

        alphabet_length = self.number_of_cnots + (len(self.alphabet_gates)-1)*self.n_qubits
        for ind, k in enumerate(range(self.number_of_cnots + (len(self.alphabet_gates)-1)*self.n_qubits)): #one hot encoding
            one_hot_gate = [-1]*alphabet_length
            one_hot_gate[ind] = 1
            self.alphabet.append(one_hot_gate)

    def load_observable(self, obs,g=1, J=0):
        """
        obs can either be a string, a list with cirq's gates or a matrix (array)
        """
        if obs == "Ising_":
            observable = [g*cirq.X.on(q) for q in self.qubits] # -J \sum_{i} Z_i Z_{i+1} - g \sum_i X_i    when g>>J
            for q in range(len(self.qubits)):
                observable.append(J*cirq.Z.on(self.qubits[q])*cirq.Z.on(self.qubits[(q+1)%len(self.qubits)]))
        else:
            print("check previous versions to load other observables.")
        return observable

    def append_to_circuit(self, one_hot_gate, circuit, params):
        """
        appends to circuit the one_hot_gate;
        and if one_hot_gate it implies a rotation,
        appends to params a symbol"""

        for ind,inst in enumerate(one_hot_gate):
            if inst == 1:  # this is faster than numpy.where
                if ind < self.number_of_cnots:
                    control, target = self.indexed_cnots[str(ind)]
                    circuit.append(self.alphabet_gates[0].on(self.qubits[control], self.qubits[target]))
                    return circuit, params
                elif self.number_of_cnots <= ind < self.number_of_cnots + self.n_qubits:
                    new_param = "th_"+str(len(params))
                    params.append(new_param)
                    circuit.append(self.alphabet_gates[1](sympy.Symbol(new_param)).on(self.qubits[int(ind%self.n_qubits)]))
                    return circuit, params
                elif self.number_of_cnots + self.n_qubits <= ind < self.number_of_cnots + 2*self.n_qubits:
                    circuit.append(self.alphabet_gates[2].on(self.qubits[int(ind%self.n_qubits)]))
                    return circuit, params
                elif self.number_of_cnots + 2*self.n_qubits <= ind < self.number_of_cnots+3*self.number_of_cnots:
                    circuit.append(self.alphabet_gates[3].on(self.qubits[int(ind%self.n_qubits)]))
                    return circuit, params
                else:
                    print("doing nothing! even not identity! careful")
                    return circuit, params
                
                
    def TFQ_model(self, symbols):
        circuit_input = tf.keras.Input(shape=(), dtype=tf.string)
        output = tfq.layers.Expectation()(
                circuit_input,
                symbol_names=symbols,
                operators=tfq.convert_to_tensor([self.observable]),
                initializer=tf.keras.initializers.RandomNormal()) #this is not strictly necessary.
        model = tf.keras.Model(inputs=circuit_input, outputs=output)
        adam = tf.keras.optimizers.Adam(learning_rate=0.01)
        model.compile(optimizer=adam, loss='mse')
        return model
        
    
    def run_circuit(self, gates_index):
        """
        takes as input vector with actions described as integer (given by RL agent),
        and outputsthe energy of that circuit (w.r.t self.observable)
            """
        circuit, symbols = [], []
        for k in gates_index:
            circuit, symbols = self.append_to_circuit(self.alphabet[k],circuit,symbols)
        tfqcircuit = tfq.convert_to_tensor([cirq.Circuit(circuit)])
        if len(params) == 0:
            expval = tfq.layers.Expectation()(
                                            tfqcircuit,
                                            operators=tfq.convert_to_tensor([self.observable]))
            energy = np.float32(np.squeeze(tf.math.reduce_sum(expval, axis=-1, keepdims=True)))
        else:
            model = self.TFQ_model(symbols)
            qoutput = tf.ones((1, 1))*self.ground_state_energy
            model.fit(x=tfqcircuit, y=qoutput, batch_size=1, epochs=150, verbose=0)
            energy = np.squeeze(tf.math.reduce_sum(model.predict(tfqcircuit), axis=-1))
            #energy = np.sum(energies)
        return energy

    

In [62]:
for k in range(2,10):
    solver = CirqSmartSolver(n_qubits=k, observable_name="Ising_")
    print(solver.number_of_cnots - solver.n_qubits*(solver.n_qubits-1))
    #print(len(solver.alphabet) - 3*solver.n_qubits - solver.number_of_cnots)
    #print("**")

In [224]:
solver = CirqSmartSolver(n_qubits=3, observable_name="Ising_")
for k in range(len(solver.alphabet)):
    circuit,_ = [], []
    circuit, _ = solver.append_to_circuit(solver.alphabet[k],circuit,_)
    print(k,"\n")
    print(cirq.Circuit(circuit))
    print("\n****\n")

0 

(0, 0): ───@───
           │
(0, 1): ───X───

****

1 

(0, 0): ───@───
           │
(0, 2): ───X───

****

2 

(0, 0): ───X───
           │
(0, 1): ───@───

****

3 

(0, 1): ───@───
           │
(0, 2): ───X───

****

4 

(0, 0): ───X───
           │
(0, 2): ───@───

****

5 

(0, 1): ───X───
           │
(0, 2): ───@───

****

6 

(0, 0): ───Ry(th_0)───

****

7 

(0, 1): ───Ry(th_0)───

****

8 

(0, 2): ───Ry(th_0)───

****

9 

(0, 0): ───Rx(-0.5π)───

****

10 

(0, 1): ───Rx(-0.5π)───

****

11 

(0, 2): ───Rx(-0.5π)───

****

12 

(0, 0): ───I───

****

13 

(0, 1): ───I───

****

14 

(0, 2): ───I───

****



In [96]:
solver = CirqSmartSolver(n_qubits=4, observable_name="Ising_")
gates_index = np.random.choice(range(len(solver.alphabet)),10)
circuit, parms = [], []

for k in gates_index:
    circuit, params = solver.append_to_circuit(solver.alphabet[k],circuit,parms)
circuit = cirq.Circuit(circuit)

In [100]:
%timeit solver.vansatz_keras_model(circuit,solver.observable,params)

36.2 ms ± 704 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [131]:
solver = CirqSmartSolver(n_qubits=4, observable_name="Ising_")
gates_index = list(np.random.choice(range(len(solver.alphabet)),10))
for k in np.arange(len(solver.alphabet))[-4:]:
    gates_index.append(k)
#gates_index.append()
#solver.run_circuit(gates_index)

In [144]:
circuit, symbols = [], []
for k in gates_index:
    circuit, symbols = solver.append_to_circuit(solver.alphabet[k],circuit,symbols)
#circuit = tfq.convert_to_tensor([cirq.Circuit(circuit)])


In [205]:
solver = CirqSmartSolver(n_qubits=4, observable_name="Ising_")
gates_index = list(np.random.choice(range(len(solver.alphabet)),10))
for k in np.arange(len(solver.alphabet))[-4:]:
    gates_index.append(k)
#gates_index.append()
solver.run_circuit(gates_index)

In [214]:
solver = CirqSmartSolver(n_qubits=3, observable_name="Ising_")
gates_index = solver.number_of_cnots + np.arange(solver.n_qubits)
solver.run_circuit(gates_index)

array(0.6932972, dtype=float32)

In [215]:
circuit, symbols = [], []
for k in gates_index:
    circuit, symbols = solver.append_to_circuit(solver.alphabet[k],circuit,symbols)

In [216]:
cirq.Circuit(circuit)

(0, 0): ───Ry(th_0)───

(0, 1): ───Ry(th_1)───

(0, 2): ───Ry(th_2)───

In [217]:
cirq.Circuit(solver.observable)

(0, 0): ───X───0j*Z((0, 0))*Z((0, 1))────────────────────────────0j*Z((0, 0))*Z((0, 2))───
               │                                                 │
(0, 1): ───X───#2───────────────────────0j*Z((0, 1))*Z((0, 2))───┼────────────────────────
                                        │                        │
(0, 2): ───X────────────────────────────#2───────────────────────#2───────────────────────